# News Rollup bot

Resources https://newsapi.org/docs/client-libraries/python and https://platform.openai.com/docs/api-reference/chat/create

First get an api key and create a .env file using example.env - NewsAPI, OpenAI

In [60]:
# Load environment variables
# pip3 install dotenv

import os
from dotenv import load_dotenv

load_dotenv()

API = os.getenv('API')
GPT = os.getenv('NEWS_GPT')

In [63]:
import requests
import json

#set variables
q = 'data analytics'
domains = 'forbes.com, venturebeat.com' # comma seperated
date_from = '2023-04-25'
date_to = '2023-04-25'
language = 'en'
sortBy = 'relevancy' # relevancy, popularity, publishedAT
pageSize = 3 # set max number of articles you'd like to summarize
page = 1

#make request
api_url = f"https://newsapi.org/v2/everything?q={q}&apiKey={API}&domains={domains}&from={date_from}&to={date_to}&language={language}&sortBy={sortBy}&pageSize={pageSize}&page={page}"
response = requests.get(api_url)
data = response.json()
print(data)

#write to tmp file
with open('tmp_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

{'status': 'ok', 'totalResults': 17, 'articles': [{'source': {'id': None, 'name': 'Forbes'}, 'author': 'David Prosser, Contributor, \n David Prosser, Contributor\n https://www.forbes.com/sites/davidprosser/', 'title': 'Unlocking Lending With Credora’s Privacy Preserving Credit Scoring', 'description': 'Credora has raised $6 million of new finance as demand for private credit continues to grow', 'url': 'https://www.forbes.com/sites/davidprosser/2023/04/25/unlocking-lending-with-credoras-privacy-preserving-credit-scoring/', 'urlToImage': 'https://imageio.forbes.com/specials-images/imageserve/644786ca26b1a43633f33129/0x0.jpg?format=jpg&crop=675,380,x0,y86,safe&width=1200', 'publishedAt': '2023-04-25T11:00:40Z', 'content': 'Darshan Vaidya, CEO of Credora\r\ncredora\r\nHow do would-be borrowers share sensitive data with potential lenders so they can secure the credit they need for business growth? US based Credora, which is … [+4609 chars]'}, {'source': {'id': None, 'name': 'Forbes'}, 'auth

In [64]:
# testing json load
# with open('tmp_data.json') as data_tmp_file:
#     data_string = data_tmp_file.read()
# data = json.loads(data_string)

# print(data)

In [67]:
# parse response for URLs
# pip3 install openai

import os
import openai
import json

# read and parse json from tmp
with open('tmp_data.json') as data_tmp_file:
    data_string = data_tmp_file.read()
data = json.loads(data_string)
    
openai.api_key = GPT

for i in data['articles']:
    article = i['url']
    content = f"Provide a summary of this article in 3 bullet points (format 1, 2, 3). Condense the response and remove fillers like 'the article' and 'the author':{article}"
    completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": content}])

    # append summary(completion) to data
    summary = completion.choices[0].message.content
    i['summary'] = summary 

    # print(completion.choices[0].message.content)

# write summarized output to file
with open('tmp_summarized_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


In [59]:
# Testing for modifying and writing out json

# print(data['articles'][0])
# print(completion.choices[0].message.content)

# summary = completion.choices[0].message.content
# data['articles'][0]['summary'] = summary

# print(data['articles'][0])

# with open('tmp_summarized_data.json', 'w', encoding='utf-8') as f:
#     json.dump(data, f, ensure_ascii=False, indent=4)


{'source': {'id': None, 'name': 'Forbes'}, 'author': 'Bernd Greifeneder, Forbes Councils Member, \n Bernd Greifeneder, Forbes Councils Member\n https://www.forbes.com/sites/forbestechcouncil/people/berndgreifeneder/', 'title': 'Getting Tool Sprawl Under Control To Enable Data-Driven Business And Cloud-Scale Growth', 'description': 'Supported by a data lakehouse, organizations could unlock the value of the petabytes—and, eventually, yottabytes—of data they have available to access more precise real-time answers to the questions that matter. This can help them not just survive but thrive …', 'url': 'https://www.forbes.com/sites/forbestechcouncil/2023/04/24/getting-tool-sprawl-under-control-to-enable-data-driven-business-and-cloud-scale-growth/', 'urlToImage': 'https://imageio.forbes.com/specials-images/imageserve/6399d93eaccb6518c9a6c001/0x0.jpg?format=jpg&width=1200', 'publishedAt': '2023-04-24T11:00:00Z', 'content': 'Bernd Greifeneder is the CTO and founder of Dynatrace, a software int